##  Обратный индекс 

Сам по себе обратный индекс не может осуществлять поиск, для этого необходимо добавить к нему определенную метрику. Это не совсем очевидная задача, поэтому немного отложим ее. А сейчас посмотрим, что полезного можно вытащить из индекса.    
По сути, индекс - это информация о частоте встречаемости слова в каждом документе.   
Из этого можно понять, например:
1. какое слово является самым часто употребимым / редким
2. какие слова встречаются всегда вместе. Так можно парсить твиттер, fb, форумы и отлавливать новые устойчивые выражения в речи
3. какой документ является самым большим / маленьким (очень изощренный способ, когда есть _len_)

### __Задача__: 
получите обратный индекс для коллекция документов.    
Перед этим постройте матрицу терм-документ и сделайте функцию булева поиска, которая по запросу будет возвращать 5 релевантных документов.   
В качестве коллекции возьмите сценарий сезонов сериала Друзья. Одна серия - один документ.

[download_friends_corpus](https://yadi.sk/d/k_M7n63A3adGSz)

Этапы:   
    1. получить коллекцию документов
    2. для каждого файла коллекции сделать необходимую на ваш взгляд предобработку
    3. получить матрицу терм-документ, написать функцию поиска по ней
    4. получить обратный индекс в виде словаря, где ключ - нормализованное слово, 
    значение - список файлов, в которых это слово встречается
    5. вывести кусочек индекса в виде таблицы 
    6. сделать анализ обратного индекса. Это задание принимается в виде кода и ответов на вопросы

![](Friends/wedding.png)

Напоминание:    
> При итерации по списку вы можете помимо самого элемента получить его порядковый номер    
``` for i, element in enumerate(your_list): ...  ```    
Иногда для получения элемента делают так -  ``` your_list[i] ```, старайтесь этого избегать

<img src="pics/inv_index3.svg" alt="Drawing" style="width: 600px;"/>

Совет для построения обратного индекса: 
> В качестве словаря используйте ``` defaultdict ``` из модуля collections   
Так можно избежать конструкции ``` dict.setdefault(key, default=None) ```

In [27]:
from collections import Counter
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from pymystem3 import Mystem
mystem = Mystem()
from string import punctuation

In [28]:
def preprocessing(input_text, del_stopwords=True, del_digit=True):
    """
    :input: raw text
    1. lowercase, del punctuation, tokenize
    2. normal form
    3. del stopwords
    4. del digits
    :return: lemmas
    """
    russian_stopwords = set(stopwords.words('russian'))
    words = [x.lower().strip(punctuation+'»«–…') for x in word_tokenize(input_text)]
    lemmas = [mystem.lemmatize(x)[0] for x in words if x]

    lemmas_arr = []
    for lemma in lemmas:
        if del_stopwords:
            if lemma in russian_stopwords:
                continue
        if del_digit:
            if lemma.isdigit():
                continue
        lemmas_arr.append(lemma)
    return lemmas_arr

In [29]:
def inverted_index(files_list):
    """
    Create inverted index by input doc collection
    :return: inverted index
    """
    global main_dir
    index = {}
    files_len = {}
    for file in files_list:
        with open(main_dir +'/'+ file, 'r', encoding='utf-8') as f:
            words = preprocessing(f.read())
            files_len[file] = len(words)
            counter = Counter(words)
            for word in counter:
                if word in index:
                    index[word][file] = counter[word]
                else:
                    index[word] = {}
                    index[word][file] = counter[word]
    return files_len, index

In [30]:
files_len, index = inverted_index(files_list)

In [99]:
import pandas as pd

pd.DataFrame(index).head()

,1.25,1.5,1.75,11б,12.50,12.75,1986ом,1999ый,2.75,20й,...,ясность,ясный,яхта,ящерица,ящик,ящичек,—,№,﻿,﻿
Friends - 1x01 - The One Where Monica Gets A Roommate.ru.txt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,16.0,NaN,1.0,NaN
Friends - 1x02 - The One With The Sonogram At The End.ru.txt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
Friends - 1x03 - The One With The Thumb.ru.txt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
Friends - 1x04 - The One With George Stephanopoulos.ru.txt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
Friends - 1x05 - The One With The East German Laundry Detergent.ru.txt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN


С помощью обратного индекса произведите следующую аналитику:  

1) общая аналитика
- какое слово является самым частотным?
- какое самым редким?
- какой набор слов есть во всех документах коллекции?

2) частота встречаемости имен главных героев в каждом сезоне      
- какой сезон был самым популярным у Чендлера? у Моники?   
- кто из главных героев статистически самый популярный? 


In [31]:
freq = []
for word in index:
    freq.append([sum(index[word].values()), word])
sorted(freq, reverse = True)[:10]

[[7517, 'это'],
 [2660, 'знать'],
 [2025, 'хотеть'],
 [1902, 'мочь'],
 [1542, 'сказать'],
 [1289, 'думать'],
 [1244, 'просто'],
 [1111, 'ладно'],
 [1071, 'давать'],
 [1035, 'твой']]

Самое частотное слово - это слово 'это'

In [40]:
sorted(freq)[150:200]

[[1, 'tяжелый'],
 [1, 'vanilla'],
 [1, 've'],
 [1, 'wan'],
 [1, 'was'],
 [1, 'we'],
 [1, 'were'],
 [1, 'whole'],
 [1, 'will'],
 [1, 'wish'],
 [1, 'with'],
 [1, 'wondering'],
 [1, 'wоw'],
 [1, 'xvid'],
 [1, 'yell'],
 [1, 'your'],
 [1, 'z'],
 [1, 'ааааааа'],
 [1, 'аааааау'],
 [1, 'ааааах'],
 [1, 'абонемент'],
 [1, 'абрикос'],
 [1, 'абсурд'],
 [1, 'абсурдный'],
 [1, 'абы'],
 [1, 'аварийный'],
 [1, 'авиа'],
 [1, 'авиамодельный'],
 [1, 'авианосец'],
 [1, 'авиапочта'],
 [1, 'австралийский'],
 [1, 'австралия'],
 [1, 'авто'],
 [1, 'автобизнес'],
 [1, 'автобусный'],
 [1, 'автомобиль'],
 [1, 'авторство'],
 [1, 'автостопом'],
 [1, 'агамемнон'],
 [1, 'агрегат'],
 [1, 'ада'],
 [1, 'адвокатский'],
 [1, 'аделаида'],
 [1, 'адидас'],
 [1, 'адно'],
 [1, 'адресат'],
 [1, 'адресовать'],
 [1, 'адский'],
 [1, 'азарт'],
 [1, 'азартный']]

Если говорить о самых редких РУССКИХ словах, то это будут слова абонемент, абрикос, абсурд - и. т.д., все они одинаково редки

In [97]:
all_docs = []
for word in index:
    if len(index[word].values()) == 165:
        all_docs.append(word)

all_docs

['просто', 'знать', 'хотеть', 'это', 'сказать', 'думать']

Такой набор слов был во всех документах коллекции (не считая стоп-слова, которые мы выкинули, я думаю, они также встречались во всех документах)

In [161]:
fr_ch = {1:0, 2:0, 3:0, 4:0, 5:0, 6:0, 7:0}
for key in list(index['чендлер'].keys()):
    for i in range(8):
        if key.startswith('Friends - '+str(i)):
            fr_ch[i]+=index['чендлер'][key]

In [162]:
fr_ch[1] = fr_ch[1]/19
fr_ch[2] = fr_ch[2]/24
fr_ch[3] = fr_ch[3]/21
fr_ch[4] = fr_ch[4]/25
fr_ch[5] = fr_ch[5]/25
fr_ch[6] = fr_ch[6]/26
fr_ch[7] = fr_ch[7]/25

In [163]:
fr_ch

{1: 3.0,
 2: 2.1666666666666665,
 3: 3.6666666666666665,
 4: 4.16,
 5: 5.52,
 6: 5.769230769230769,
 7: 5.76}

Самый популярный сезон у Чендлера - 6

In [150]:
fr_m = {1:0, 2:0, 3:0, 4:0, 5:0, 6:0, 7:0}
for key in list(index['моника'].keys()):
    for i in range(8):
        if key.startswith('Friends - '+str(i)):
            fr_m[i]+=index['моника'][key]

In [153]:
fr_m[1] = fr_m[1]/19
fr_m[2] = fr_m[2]/24
fr_m[3] = fr_m[3]/21
fr_m[4] = fr_m[4]/25
fr_m[5] = fr_m[5]/25
fr_m[6] = fr_m[6]/26
fr_m[7] = fr_m[7]/25

Я подумала что будет честно разделить на количество серий в сезоне, т.к. больше серий = больше упоминаний, и это не чистая "популярность"

In [154]:
fr_m

{1: 3.0,
 2: 3.25,
 3: 3.2857142857142856,
 4: 2.88,
 5: 5.12,
 6: 4.730769230769231,
 7: 6.08}

Итого, самый популярный сезон Моники - седьмой

In [102]:
friends = ['моника', 'росс', 'джоуи', 'чендлер', 'фиби', 'рэйчел']
freq = {}

for friend in friends:
    freq[friend] = sum(index[friend].values())
freq

{'джоуи': 681,
 'моника': 679,
 'росс': 1013,
 'рэйчел': 236,
 'фиби': 574,
 'чендлер': 722}

Выходит, что самый популярный - Росс.

## Функция ранжирования Okapi BM25

Для обратного индекса есть общепринятая формула для ранжирования *Okapi best match 25* ([Okapi BM25](https://ru.wikipedia.org/wiki/Okapi_BM25)).    
Пусть дан запрос $Q$, содержащий слова  $q_1, ... , q_n$, тогда функция BM25 даёт следующую оценку релевантности документа $D$ запросу $Q$:

$$ score(D, Q) = \sum_{i}^{n} \text{IDF}(q_i)*\frac{(k_1+1)*f(q_i,D)}{f(q_i,D)+k_1(1-b+b\frac{|D|}{avgdl})} $$ 
где   
>$f(q_i,D)$ - частота слова $q_i$ в документе $D$ (TF)       
$|D|$ - длина документа (количество слов в нём)   
*avgdl* — средняя длина документа в коллекции    
$k_1$ и $b$ — свободные коэффициенты, обычно их выбирают как $k_1$=2.0 и $b$=0.75   
$$$$
$\text{IDF}(q_i)$ есть обратная документная частота (IDF) слова $q_i$: 
$$\text{IDF}(q_i) = \log\frac{N-n(q_i)+0.5}{n(q_i)+0.5},$$
>> где $N$ - общее количество документов в коллекции   
$n(q_i)$ — количество документов, содержащих $q_i$

In [45]:
### реализуйте эту функцию ранжирования 
import math
from math import log

k1 = 2.0
b = 0.75

def score_BM25(qf, dl, avgdl, k1, b, N, n) -> float:
    """
    Compute similarity score between search query and documents from collection
    :return: score
    """
    score = math.log((N-n+0.5)/(n+0.5)) * (k1+1)*qf/(qf+k1*(1-b+b*(dl/avgdl)))
    return score

### __Задача__:    
напишите функцию, которая сортирует поисковую выдачу для любого входящего запроса согласно метрике *Okapi BM25*.    
Выведите 10 первых результатов и их скор по запросу **рождественские каникулы**. 

In [87]:
import operator

def compute_sim(word, index, files_len):
    """
    Compute similarity score between search query and documents from collection
    :return: score
    """
    N = len(files_len)
    avgdl = sum(files_len.values())/N
    if word in index:
        n = len(index[word])
        result = {}
        for file in index[word]:
            qf = index[word][file]
            score = score_BM25(qf, files_len[file], avgdl, k1, b, N, n)
            result[file] = score
        return result
    else:
        return {}


def get_search_result(inquiry):
    """
    Compute sim score between search query and all documents in collection
    :return: list of files
    """
    global index, files_len
    score = defaultdict(int)
    words = preprocessing(inquiry)
    for word in words:
        result = compute_sim(word, index, files_len)
        for file in result:
            score[file] += result[file]
    return sorted(score.items(), key=operator.itemgetter(1), reverse = True)[:10]

In [88]:
from collections import defaultdict
get_search_result('рождественские каникулы')

[('Friends - 7x10 - The One With The Holiday Armadillo.ru.txt',
  9.776309773280323),
 ("Friends - 6x19 - The One With Joey's Fridge.ru.txt", 7.831350385518332),
 ('Friends - 3x10 - The One Where Rachel Quits.ru.txt', 5.6005292157657385),
 ("Friends - 2x09 - The One With Phoebe's Dad.ru.txt", 4.787759807254001),
 ('Friends - 1x17 - The One With Two Parts (2).ru.txt', 4.139458794305614),
 ("Friends - 4x03 - The One With The 'Cuffs.ru.txt", 4.121790362729642),
 ('Friends - 1x16 - The One With Two Parts (1).ru.txt', 4.052599599125035),
 ('Friends - 4x10 - The One With The Girl From Poughkeepsie.ru.txt',
  4.02648492884397),
 ('Friends - 6x12 - The One With The Joke.ru.txt', 3.4614800887510877),
 ('Friends - 6x09 - The One Where Ross Got High.ru.txt', 3.4162272692841538)]